In [ ]:
import ibis

In [ ]:
#### disable progress bar to show up
ibis.get_backend().raw_sql("PRAGMA disable_progress_bar;");

In [ ]:
from ibis import _

In [ ]:
ibis.options.interactive = True

# Data from source cooperative

**Shoutout:** Matt Forrest

Download the data from (it's a 2.3 GB parquet file): [https://beta.source.coop/vida/google-microsoft-open-buildings/geoparquet/by_country/country_iso=ARG/ARG.parquet]("https://beta.source.coop/vida/google-microsoft-open-buildings/geoparquet/by_country/country_iso=ARG/ARG.parquet")



In [ ]:
t = ibis.read_parquet("ARG.parquet")

In [ ]:
t.schema()

In [ ]:
# number of rows 
t.count() / 1e6

In [ ]:
t.describe()

In [ ]:
t.geometry

# Filter the building within a certain radius of Córdoba city

In [ ]:
cordoba_lat = -31.4135
cordoba_lon = -64.18105

In [ ]:
import shapely

In [ ]:
cba = shapely.Point(cordoba_lon, cordoba_lat)

In [ ]:
cba

In [ ]:
cba_buildings = t.filter(t.geometry.d_within(cba, 0.1))
cba_buildings

In [ ]:
#cache this to not do the filter everytime
cba_buildings = cba_buildings.cache()

# Let's plot 

In [ ]:
from lonboard import Map, ScatterplotLayer, PolygonLayer

In [ ]:
cba_buildings.count()

## To geopandas to plot with lonboard 

### (hoping that during the sprints we can plot directly from an Ibis table)


In [ ]:
# to geopandas to plot (hoping that during the sprints we can plot directly from an Ibis table)
cba_bd_gdf = cba_buildings.to_pandas()

In [ ]:
cba_bd_gdf.crs = "EPSG:4326"

In [ ]:
cba_ctr_gdf = ibis.literal(cba, "geometry").as_table().to_pandas()
cba_ctr_gdf.crs = "EPSG:4326"
type(cba_ctr_gdf)

In [ ]:
cba_ctr = ScatterplotLayer.from_geopandas(
    cba_ctr_gdf, get_fill_color="red", get_radius=100
)

building_layer = PolygonLayer.from_geopandas(
    cba_bd_gdf, get_fill_color="blue", opacity=0.1
)

In [ ]:
m1 = Map(
    [
    cba_ctr,
    building_layer, 
    ],
    view_state={"longitude": -64.18105, "latitude": -31.4135, "zoom": 10}
)
m1

In [ ]:
#long, lat university of Cordoba =  -64.185999256 , -31.43583159

In [ ]:
unc =  ibis.literal(shapely.Point(-64.185999256, -31.43583159), "geometry")

In [ ]:
unc_buf = unc.buffer(0.01)

In [ ]:
unc_buf

In [ ]:
unc_buildings = cba_buildings.filter(_.geometry.within(unc_buf))

In [ ]:
unc_buildings.count()

In [ ]:
unc_gdf = unc_buildings.to_pandas()

In [ ]:
unc_gdf.crs = "EPSG:4326"

In [ ]:
unc_buf_gdf = unc_buf.as_table().to_pandas()
unc_buf_gdf.crs = "EPSG:4326"

In [ ]:
unc_layer = PolygonLayer.from_geopandas(
    unc_gdf, get_fill_color="red", opacity=0.1
)

unc_buf_layer = PolygonLayer.from_geopandas(
    unc_buf_gdf, get_fill_color="blue", opacity=0.005, 
)

In [ ]:
m2 = Map(
    [
    unc_buf_layer, 
    unc_layer,
    ],
    view_state={"longitude": -64.185999256, "latitude": -31.43583159, "zoom": 10}
)
m2

In [ ]:
con = ibis.get_backend()

In [ ]:
con.raw_sql(f"COPY ({ibis.to_sql(unc_buildings)}) TO 'unc_buildings.geojson' WITH (FORMAT GDAL, DRIVER 'GeoJSON', LAYER_CREATION_OPTIONS 'WRITE_BBOX=YES')")

In VSCode install the Geo Data Viewer extension, then right click on `unc_buildings.geojson` and select view map to see it in csvcode. 